In [1]:
import requests
import pandas as pd
import time
import json
import ast
import numpy as np
import glob
import datetime

In [2]:
from ta import trend
from ta import momentum

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
def convert_to_json(text):
    try:
        return ast.literal_eval(text)
    except:
#         print(text)
        return {}

In [5]:
def read_all_file_in_directory(directory, file_type, sep = "\t"):
    print(directory)
    file_list= [f for f in glob.glob(directory + "*." + file_type)]
    
    all_data = pd.DataFrame()
    temp = pd.DataFrame()
    for file in file_list:
            df = pd.read_csv(file, sep= sep, encoding = "utf8", engine='python')
            temp = pd.concat([temp, df])
    all_data = pd.concat([all_data, temp])
    return all_data

In [6]:
def clean_candle_data(df, price_type = 'ask' ):
    
    df_modified = df[['complete', 'currency_pair', 'time', 'volume', price_type]]
    df_modified[price_type] = df_modified[price_type].astype(str)
    df_modified[price_type] = df_modified[price_type].apply(convert_to_json)
    
    price_df = pd.json_normalize(df_modified[price_type])
    
    df_modified = pd.merge(df_modified, price_df,
                          left_index=True, right_index=True)
    
    df_modified = df_modified[['complete','currency_pair', 'time', 'volume','o', 'h', 'l', 'c']]
    df_modified.columns = ['complete','currency_pair', 'time', 'volume','price_open','price_high', 'price_low', 'price_close']
    
    df_modified[["price_open", "price_high", "price_low", "price_close"]] = df_modified[["price_open", "price_high",
                                                                                         "price_low", "price_close"]].apply(pd.to_numeric)
    df_modified["time"] = pd.to_datetime(df_modified["time"])
    
    return df_modified

In [7]:
def get_macd_indicator(df, window_slow: int = 26, window_fast: int = 12, window_sign: int = 9, fillna: bool = False):
    # Innitialising MACD indicator
    indicator_macd = trend.MACD(close = df["price_close"])
    # Create columns for MACCD line, singal line and macd histogram (macd crosses signal line)
    df['macd'] = indicator_macd.macd()
    df['macd_signal'] = indicator_macd.macd_signal()
    df['macd_signal_diff'] = indicator_macd.macd_diff()
    return df

#### READ AND CLEAN PRICE DATA

In [10]:
directory = "/Users/nguyenhoangnam/Documents/Python/Stock - Forex Investing & Trading/Forex/OANDA/"
curency_pair = "EURUSD"
time_range = "M1"
path = directory + curency_pair + "/" +  time_range + "/Raw Data/"


df = read_all_file_in_directory(path, file_type = "csv")
df.shape

/Users/nguyenhoangnam/Documents/Python/Stock - Forex Investing & Trading/Forex/OANDA/EURUSD/M1/Raw Data/


(120000, 6)

In [11]:
df_clean = clean_candle_data(df)

In [12]:
df_clean = df_clean.head(10000)

In [59]:
activity_dict = {}
for index in range(500, len(df_clean)):
    df_modifed = df_clean.head(index)
    df_with_indicators =  get_macd_indicator(df_modifed)
    
    indicator_sma = trend.SMAIndicator(close = df_with_indicators["price_close"], window=200)
    df_with_indicators['sma_200'] = indicator_sma.sma_indicator()

    indicator_rsi = momentum.RSIIndicator(close = df_with_indicators["price_close"], window=14)
    df_with_indicators['rsi_14'] = indicator_rsi.rsi()
    
    df_with_indicators_complete = df_with_indicators[df_with_indicators["complete"] == True]
    
    if ((df_with_indicators_complete["macd_signal_diff"].iloc[-1] < 0) and
        (df_with_indicators_complete["macd_signal_diff"].iloc[-2] < 0) and
        (df_with_indicators_complete["macd_signal_diff"].iloc[-3] >= 0) and
        (df_with_indicators_complete["rsi_14"].iloc[-1] >= 30) and
        (df_with_indicators_complete["time"].iloc[-1].time() >= datetime.time(6, 00)) and
        (df_with_indicators_complete["time"].iloc[-1].time() <= datetime.time(18, 00))):
        decision = "SELL"
        print(df_with_indicators_complete["time"].iloc[-1], "SELL")
        

    elif ((df_with_indicators_complete["macd_signal_diff"].iloc[-1] > 0) and
         (df_with_indicators_complete["macd_signal_diff"].iloc[-2] > 0) and
         (df_with_indicators_complete["macd_signal_diff"].iloc[-3] <= 0) and
         (df_with_indicators_complete["rsi_14"].iloc[-1] <= 70 ) and
         (df_with_indicators_complete["time"].iloc[-1].time() >= datetime.time(6, 00)) and
         (df_with_indicators_complete["time"].iloc[-1].time() <= datetime.time(18, 00))):
        decision = "BUY"
        print(df_with_indicators_complete["time"].iloc[-1], "BUY")

    else:
        print(df_with_indicators_complete["time"].iloc[-1], "Nothing happens")
        decision = "NOTHING HAPPENS"
    activity_dict[df_with_indicators_complete["time"].iloc[-1]] = decision


2021-01-04 06:46:00+00:00 Nothing happens
2021-01-04 06:47:00+00:00 Nothing happens
2021-01-04 06:48:00+00:00 Nothing happens
2021-01-04 06:49:00+00:00 Nothing happens
2021-01-04 06:50:00+00:00 Nothing happens
2021-01-04 06:51:00+00:00 Nothing happens
2021-01-04 06:52:00+00:00 Nothing happens
2021-01-04 06:53:00+00:00 Nothing happens
2021-01-04 06:54:00+00:00 Nothing happens
2021-01-04 06:55:00+00:00 BUY
2021-01-04 06:56:00+00:00 Nothing happens
2021-01-04 06:57:00+00:00 Nothing happens
2021-01-04 06:58:00+00:00 Nothing happens
2021-01-04 06:59:00+00:00 Nothing happens
2021-01-04 07:00:00+00:00 Nothing happens
2021-01-04 07:01:00+00:00 Nothing happens
2021-01-04 07:02:00+00:00 Nothing happens
2021-01-04 07:03:00+00:00 Nothing happens
2021-01-04 07:04:00+00:00 Nothing happens
2021-01-04 07:05:00+00:00 Nothing happens
2021-01-04 07:06:00+00:00 Nothing happens
2021-01-04 07:07:00+00:00 Nothing happens
2021-01-04 07:08:00+00:00 Nothing happens
2021-01-04 07:09:00+00:00 Nothing happens
2021

2021-01-04 10:32:00+00:00 Nothing happens
2021-01-04 10:33:00+00:00 Nothing happens
2021-01-04 10:34:00+00:00 Nothing happens
2021-01-04 10:35:00+00:00 Nothing happens
2021-01-04 10:36:00+00:00 Nothing happens
2021-01-04 10:37:00+00:00 Nothing happens
2021-01-04 10:38:00+00:00 Nothing happens
2021-01-04 10:39:00+00:00 Nothing happens
2021-01-04 10:40:00+00:00 Nothing happens
2021-01-04 10:41:00+00:00 BUY
2021-01-04 10:42:00+00:00 Nothing happens
2021-01-04 10:43:00+00:00 Nothing happens
2021-01-04 10:44:00+00:00 Nothing happens
2021-01-04 10:45:00+00:00 Nothing happens
2021-01-04 10:46:00+00:00 Nothing happens
2021-01-04 10:47:00+00:00 Nothing happens
2021-01-04 10:48:00+00:00 Nothing happens
2021-01-04 10:49:00+00:00 Nothing happens
2021-01-04 10:50:00+00:00 Nothing happens
2021-01-04 10:51:00+00:00 Nothing happens
2021-01-04 10:52:00+00:00 Nothing happens
2021-01-04 10:53:00+00:00 Nothing happens
2021-01-04 10:54:00+00:00 SELL
2021-01-04 10:55:00+00:00 Nothing happens
2021-01-04 10:5

2021-01-04 14:14:00+00:00 BUY
2021-01-04 14:15:00+00:00 Nothing happens
2021-01-04 14:16:00+00:00 Nothing happens
2021-01-04 14:17:00+00:00 SELL
2021-01-04 14:18:00+00:00 Nothing happens
2021-01-04 14:19:00+00:00 Nothing happens
2021-01-04 14:20:00+00:00 Nothing happens
2021-01-04 14:21:00+00:00 BUY
2021-01-04 14:22:00+00:00 Nothing happens
2021-01-04 14:23:00+00:00 Nothing happens
2021-01-04 14:24:00+00:00 Nothing happens
2021-01-04 14:25:00+00:00 Nothing happens
2021-01-04 14:26:00+00:00 Nothing happens
2021-01-04 14:27:00+00:00 Nothing happens
2021-01-04 14:28:00+00:00 Nothing happens
2021-01-04 14:29:00+00:00 Nothing happens
2021-01-04 14:30:00+00:00 Nothing happens
2021-01-04 14:31:00+00:00 Nothing happens
2021-01-04 14:32:00+00:00 Nothing happens
2021-01-04 14:33:00+00:00 Nothing happens
2021-01-04 14:34:00+00:00 SELL
2021-01-04 14:35:00+00:00 Nothing happens
2021-01-04 14:36:00+00:00 Nothing happens
2021-01-04 14:37:00+00:00 Nothing happens
2021-01-04 14:38:00+00:00 Nothing happ

2021-01-04 18:03:00+00:00 Nothing happens
2021-01-04 18:04:00+00:00 Nothing happens
2021-01-04 18:05:00+00:00 Nothing happens
2021-01-04 18:06:00+00:00 Nothing happens
2021-01-04 18:07:00+00:00 Nothing happens
2021-01-04 18:08:00+00:00 Nothing happens
2021-01-04 18:09:00+00:00 Nothing happens
2021-01-04 18:10:00+00:00 Nothing happens
2021-01-04 18:11:00+00:00 Nothing happens
2021-01-04 18:12:00+00:00 Nothing happens
2021-01-04 18:13:00+00:00 Nothing happens
2021-01-04 18:14:00+00:00 Nothing happens
2021-01-04 18:15:00+00:00 Nothing happens
2021-01-04 18:16:00+00:00 Nothing happens
2021-01-04 18:17:00+00:00 Nothing happens
2021-01-04 18:18:00+00:00 Nothing happens
2021-01-04 18:19:00+00:00 Nothing happens
2021-01-04 18:20:00+00:00 Nothing happens
2021-01-04 18:21:00+00:00 Nothing happens
2021-01-04 18:22:00+00:00 Nothing happens
2021-01-04 18:23:00+00:00 Nothing happens
2021-01-04 18:24:00+00:00 Nothing happens
2021-01-04 18:25:00+00:00 Nothing happens
2021-01-04 18:26:00+00:00 Nothing 

2021-01-04 21:21:00+00:00 Nothing happens
2021-01-04 21:22:00+00:00 Nothing happens
2021-01-04 21:23:00+00:00 Nothing happens
2021-01-04 21:24:00+00:00 Nothing happens
2021-01-04 21:25:00+00:00 Nothing happens
2021-01-04 21:26:00+00:00 Nothing happens
2021-01-04 21:27:00+00:00 Nothing happens
2021-01-04 21:28:00+00:00 Nothing happens
2021-01-04 21:29:00+00:00 Nothing happens
2021-01-04 21:30:00+00:00 Nothing happens
2021-01-04 21:31:00+00:00 Nothing happens
2021-01-04 21:32:00+00:00 Nothing happens
2021-01-04 21:33:00+00:00 Nothing happens
2021-01-04 21:34:00+00:00 Nothing happens
2021-01-04 21:35:00+00:00 Nothing happens
2021-01-04 21:36:00+00:00 Nothing happens
2021-01-04 21:37:00+00:00 Nothing happens
2021-01-04 21:38:00+00:00 Nothing happens
2021-01-04 21:39:00+00:00 Nothing happens
2021-01-04 21:40:00+00:00 Nothing happens
2021-01-04 21:41:00+00:00 Nothing happens
2021-01-04 21:42:00+00:00 Nothing happens
2021-01-04 21:43:00+00:00 Nothing happens
2021-01-04 21:44:00+00:00 Nothing 

...

2021-01-12 07:04:00+00:00 Nothing happens
2021-01-12 07:05:00+00:00 Nothing happens
2021-01-12 07:06:00+00:00 Nothing happens
2021-01-12 07:07:00+00:00 Nothing happens
2021-01-12 07:08:00+00:00 Nothing happens
2021-01-12 07:09:00+00:00 Nothing happens
2021-01-12 07:10:00+00:00 Nothing happens
2021-01-12 07:11:00+00:00 Nothing happens
2021-01-12 07:12:00+00:00 Nothing happens
2021-01-12 07:13:00+00:00 BUY
2021-01-12 07:14:00+00:00 Nothing happens
2021-01-12 07:15:00+00:00 Nothing happens
2021-01-12 07:16:00+00:00 BUY
2021-01-12 07:17:00+00:00 Nothing happens
2021-01-12 07:18:00+00:00 Nothing happens
2021-01-12 07:19:00+00:00 Nothing happens
2021-01-12 07:20:00+00:00 Nothing happens
2021-01-12 07:21:00+00:00 Nothing happens
2021-01-12 07:22:00+00:00 Nothing happens
2021-01-12 07:23:00+00:00 Nothing happens
2021-01-12 07:24:00+00:00 Nothing happens
2021-01-12 07:25:00+00:00 Nothing happens
2021-01-12 07:26:00+00:00 Nothing happens
2021-01-12 07:27:00+00:00 Nothing happens
2021-01-12 07:28

2021-01-12 10:30:00+00:00 Nothing happens
2021-01-12 10:31:00+00:00 Nothing happens
2021-01-12 10:32:00+00:00 Nothing happens
2021-01-12 10:33:00+00:00 Nothing happens
2021-01-12 10:34:00+00:00 Nothing happens
2021-01-12 10:35:00+00:00 Nothing happens
2021-01-12 10:36:00+00:00 Nothing happens
2021-01-12 10:37:00+00:00 Nothing happens
2021-01-12 10:38:00+00:00 Nothing happens
2021-01-12 10:39:00+00:00 Nothing happens
2021-01-12 10:40:00+00:00 Nothing happens
2021-01-12 10:41:00+00:00 Nothing happens
2021-01-12 10:42:00+00:00 Nothing happens
2021-01-12 10:43:00+00:00 Nothing happens
2021-01-12 10:44:00+00:00 Nothing happens
2021-01-12 10:45:00+00:00 Nothing happens
2021-01-12 10:46:00+00:00 Nothing happens
2021-01-12 10:47:00+00:00 Nothing happens
2021-01-12 10:48:00+00:00 Nothing happens
2021-01-12 10:49:00+00:00 Nothing happens
2021-01-12 10:50:00+00:00 Nothing happens
2021-01-12 10:51:00+00:00 Nothing happens
2021-01-12 10:52:00+00:00 Nothing happens
2021-01-12 10:53:00+00:00 Nothing 

2021-01-12 14:03:00+00:00 Nothing happens
2021-01-12 14:04:00+00:00 Nothing happens
2021-01-12 14:05:00+00:00 SELL
2021-01-12 14:06:00+00:00 Nothing happens
2021-01-12 14:07:00+00:00 Nothing happens
2021-01-12 14:08:00+00:00 Nothing happens
2021-01-12 14:09:00+00:00 Nothing happens
2021-01-12 14:10:00+00:00 Nothing happens
2021-01-12 14:11:00+00:00 BUY
2021-01-12 14:12:00+00:00 Nothing happens
2021-01-12 14:13:00+00:00 Nothing happens
2021-01-12 14:14:00+00:00 Nothing happens
2021-01-12 14:15:00+00:00 Nothing happens
2021-01-12 14:16:00+00:00 Nothing happens
2021-01-12 14:17:00+00:00 Nothing happens
2021-01-12 14:18:00+00:00 Nothing happens
2021-01-12 14:19:00+00:00 Nothing happens
2021-01-12 14:20:00+00:00 Nothing happens
2021-01-12 14:21:00+00:00 Nothing happens
2021-01-12 14:22:00+00:00 Nothing happens
2021-01-12 14:23:00+00:00 Nothing happens
2021-01-12 14:24:00+00:00 Nothing happens
2021-01-12 14:25:00+00:00 Nothing happens
2021-01-12 14:26:00+00:00 Nothing happens
2021-01-12 14:2

2021-01-12 17:27:00+00:00 Nothing happens
2021-01-12 17:28:00+00:00 Nothing happens
2021-01-12 17:29:00+00:00 Nothing happens
2021-01-12 17:30:00+00:00 Nothing happens
2021-01-12 17:31:00+00:00 Nothing happens
2021-01-12 17:32:00+00:00 Nothing happens
2021-01-12 17:33:00+00:00 Nothing happens
2021-01-12 17:34:00+00:00 Nothing happens
2021-01-12 17:35:00+00:00 Nothing happens
2021-01-12 17:36:00+00:00 Nothing happens
2021-01-12 17:37:00+00:00 Nothing happens
2021-01-12 17:38:00+00:00 Nothing happens
2021-01-12 17:39:00+00:00 Nothing happens
2021-01-12 17:40:00+00:00 SELL
2021-01-12 17:41:00+00:00 Nothing happens
2021-01-12 17:42:00+00:00 Nothing happens
2021-01-12 17:43:00+00:00 Nothing happens
2021-01-12 17:44:00+00:00 Nothing happens
2021-01-12 17:45:00+00:00 Nothing happens
2021-01-12 17:46:00+00:00 Nothing happens
2021-01-12 17:47:00+00:00 Nothing happens
2021-01-12 17:48:00+00:00 Nothing happens
2021-01-12 17:49:00+00:00 Nothing happens
2021-01-12 17:50:00+00:00 Nothing happens
202

2021-01-12 21:04:00+00:00 Nothing happens
2021-01-12 21:05:00+00:00 Nothing happens
2021-01-12 21:06:00+00:00 Nothing happens
2021-01-12 21:07:00+00:00 Nothing happens
2021-01-12 21:08:00+00:00 Nothing happens
2021-01-12 21:09:00+00:00 Nothing happens
2021-01-12 21:10:00+00:00 Nothing happens
2021-01-12 21:11:00+00:00 Nothing happens
2021-01-12 21:12:00+00:00 Nothing happens
2021-01-12 21:13:00+00:00 Nothing happens
2021-01-12 21:14:00+00:00 Nothing happens
2021-01-12 21:15:00+00:00 Nothing happens
2021-01-12 21:16:00+00:00 Nothing happens
2021-01-12 21:17:00+00:00 Nothing happens
2021-01-12 21:18:00+00:00 Nothing happens
2021-01-12 21:19:00+00:00 Nothing happens
2021-01-12 21:20:00+00:00 Nothing happens
2021-01-12 21:21:00+00:00 Nothing happens
2021-01-12 21:22:00+00:00 Nothing happens
2021-01-12 21:23:00+00:00 Nothing happens
2021-01-12 21:24:00+00:00 Nothing happens
2021-01-12 21:25:00+00:00 Nothing happens
2021-01-12 21:26:00+00:00 Nothing happens
2021-01-12 21:27:00+00:00 Nothing 

In [60]:
activity_df = pd.DataFrame(list(activity_dict.items()),columns = ['time','order_type']) 

In [61]:
df_clean_order_type = pd.merge(df_clean, activity_df,
                              on = "time", how = "left").fillna("NOTHING HAPPENS")

In [62]:
df_clean_order_type['order_type'].value_counts()

NOTHING HAPPENS    9657
SELL                173
BUY                 170
Name: order_type, dtype: int64

In [63]:
df_clean_order_type

,complete,currency_pair,time,volume,price_open,price_high,price_low,price_close,order_type
0,True,EUR_USD,2021-01-03 22:00:00+00:00,7,1.22415,1.22430,1.22414,1.22423,NOTHING HAPPENS
1,True,EUR_USD,2021-01-03 22:01:00+00:00,7,1.22415,1.22460,1.22414,1.22452,NOTHING HAPPENS
2,True,EUR_USD,2021-01-03 22:02:00+00:00,5,1.22452,1.22453,1.22435,1.22435,NOTHING HAPPENS
3,True,EUR_USD,2021-01-03 22:03:00+00:00,2,1.22433,1.22433,1.22425,1.22425,NOTHING HAPPENS
4,True,EUR_USD,2021-01-03 22:04:00+00:00,11,1.22406,1.22417,1.22398,1.22413,NOTHING HAPPENS
...,...,...,...,...,...,...,...,...,...
9995,True,EUR_USD,2021-01-12 23:37:00+00:00,10,1.22078,1.22081,1.22078,1.22078,NOTHING HAPPENS
9996,True,EUR_USD,2021-01-12 23:38:00+00:00,15,1.22080,1.22080,1.22070,1.22074,NOTHING HAPPENS
9997,True,EUR_USD,2021-01-12 23:39:00+00:00,11,1.22076,1.22085,1.22076,1.22085,NOTHING HAPPENS
9998,True,EUR_USD,2021-01-12 23:40:00+00:00,5,1.22086,1.22087,1.22081,1.22081,NOTHING HAPPENS


In [64]:
def trade_execution(df, order_type, order_time, take_profit_pip = 10, stop_loss_pip = 10):
    
    df["index"] = df.index
    signal_time_index = df.loc[df['time'] == order_time, 'index'].iloc[0]
    order_time_index = signal_time_index + 1
    order_price_start = df["price_open"][order_time_index]
    order_time_start = df["time"][order_time_index]

    if order_type == "BUY":
        price_tp = round(order_price_start + (take_profit_pip/10000),5)
        price_sl = round(order_price_start - (stop_loss_pip/10000),5)

        for index in range(order_time_index, len(df)):
            current_price_low = round(df["price_low"][index],5)
            current_price_high = round(df["price_high"][index],5)
            current_time = df["time"][index]

            if (current_price_low <= price_sl) and (current_price_high >= price_tp):
                result_df = pd.DataFrame({'order_type': order_type,
                                          'order_time_start': order_time_start,
                                          'order_time_end': current_time,
                                          'order_price_start': order_price_start,
                                          'order_price_sl': price_sl,
                                          'order_price_tp': price_tp,
                                          'current_price_high': current_price_high,
                                          'current_price_low': current_price_low,
                                          'order_outcome': "Not Determined"}, index=[0])
                return result_df
                break
            elif (current_price_low <= price_sl):
                result_df = pd.DataFrame({'order_type': order_type,
                                          'order_time_start': order_time_start,
                                          'order_time_end': current_time,
                                          'order_price_start': order_price_start,
                                          'order_price_sl': price_sl,
                                          'order_price_tp': price_tp,
                                          'current_price_high': current_price_high,
                                          'current_price_low': current_price_low,
                                          'order_outcome': "Stop Loss Order Executed"}, index=[0])
                return result_df
                break
            elif (current_price_high >= price_tp):
                result_df = pd.DataFrame({'order_type': order_type,
                                          'order_time_start': order_time_start,
                                          'order_time_end': current_time,
                                          'order_price_start': order_price_start,
                                          'order_price_sl': price_sl,
                                          'order_price_tp': price_tp,
                                          'current_price_high': current_price_high,
                                          'current_price_low': current_price_low,
                                          'order_outcome': "Take Profit Order Executed"}, index=[0])
                return result_df
                break
            else:
                continue

    elif order_type == "SELL":
        price_tp = round(order_price_start - (take_profit_pip/10000),5)
        price_sl = round(order_price_start + (stop_loss_pip/10000),5)
#         print(order_type, order_time_start, "start:" + str(order_price_start), "tp:" + str(price_tp), "sl:" + str(price_sl) )

        for index in range(order_time_index, len(df)):
            current_price_low = round(df["price_low"][index],5)
            current_price_high = round(df["price_high"][index],5)
            current_time = df["time"][index]
#             print(order_type, current_time, "start:" + str(order_price_start), "low:" + str(current_price_low), "high:" + str(current_price_high) )

            if (current_price_low <= price_tp) and (current_price_high >= price_sl):
                result_df = pd.DataFrame({'order_type': order_type,
                                          'order_time_start': order_time_start,
                                          'order_time_end': current_time,
                                          'order_price_start': order_price_start,
                                          'order_price_sl': price_sl,
                                          'order_price_tp': price_tp,
                                          'current_price_high': current_price_high,
                                          'current_price_low': current_price_low,
                                          'order_outcome': "Not Determined"}, index=[0])
                return result_df
                break
            elif (current_price_low <= price_tp):
                result_df = pd.DataFrame({'order_type': order_type,
                                          'order_time_start': order_time_start,
                                          'order_time_end': current_time,
                                          'order_price_start': order_price_start,
                                          'order_price_sl': price_sl,
                                          'order_price_tp': price_tp,
                                          'current_price_high': current_price_high,
                                          'current_price_low': current_price_low,
                                          'order_outcome': "Take Profit Order Executed"}, index=[0])
                return result_df
                break
            elif (current_price_high >= price_sl):
                result_df = pd.DataFrame({'order_type': order_type,
                                          'order_time_start': order_time_start,
                                          'order_time_end': current_time,
                                          'order_price_start': order_price_start,
                                          'order_price_sl': price_sl,
                                          'order_price_tp': price_tp,
                                          'current_price_high': current_price_high,
                                          'current_price_low': current_price_low,
                                          'order_outcome': "Stop Loss Order Executed"}, index=[0])
                return result_df
                break
            else:
                continue
    else:
        return("Invalid Order Type (only buy or sell)")

    

In [65]:
def get_trade_results(df, take_profit_pip = 5, stop_loss_pip = 5):

    df_with_signals_filtered = df[df["order_type"].isin(["BUY", "SELL"])].reset_index(drop=True)
    print(df_with_signals_filtered.shape)

    

    output_df = pd.DataFrame()
    for i in range(0,len(df_with_signals_filtered)):
        output = trade_execution(df, df_with_signals_filtered["order_type"][i], df_with_signals_filtered["time"][i],take_profit_pip,stop_loss_pip)
        output_df = output_df.append(output)
    return output_df
    




In [66]:
test = get_trade_results(df_clean_order_type)

(343, 9)


In [67]:
pd.DataFrame(test["order_outcome"].value_counts())

,order_outcome
Take Profit Order Executed,177
Stop Loss Order Executed,161
Not Determined,5


In [68]:
df_with_indicators_complete

,complete,currency_pair,time,volume,price_open,price_high,price_low,price_close,macd,macd_signal,macd_signal_diff,sma_200,rsi_14
0,True,EUR_USD,2021-01-03 22:00:00+00:00,7,1.22415,1.22430,1.22414,1.22423,NaN,NaN,NaN,NaN,NaN
1,True,EUR_USD,2021-01-03 22:01:00+00:00,7,1.22415,1.22460,1.22414,1.22452,NaN,NaN,NaN,NaN,NaN
2,True,EUR_USD,2021-01-03 22:02:00+00:00,5,1.22452,1.22453,1.22435,1.22435,NaN,NaN,NaN,NaN,NaN
3,True,EUR_USD,2021-01-03 22:03:00+00:00,2,1.22433,1.22433,1.22425,1.22425,NaN,NaN,NaN,NaN,NaN
4,True,EUR_USD,2021-01-03 22:04:00+00:00,11,1.22406,1.22417,1.22398,1.22413,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9994,True,EUR_USD,2021-01-12 23:36:00+00:00,17,1.22077,1.22080,1.22071,1.22080,0.000024,0.000012,0.000012,1.220598,55.920718
9995,True,EUR_USD,2021-01-12 23:37:00+00:00,10,1.22078,1.22081,1.22078,1.22078,0.000027,0.000015,0.000012,1.220602,54.735902
9996,True,EUR_USD,2021-01-12 23:38:00+00:00,15,1.22080,1.22080,1.22070,1.22074,0.000026,0.000017,0.000009,1.220605,52.347070
9997,True,EUR_USD,2021-01-12 23:39:00+00:00,11,1.22076,1.22085,1.22076,1.22085,0.000034,0.000020,0.000014,1.220610,57.801258


In [69]:
test = df_with_indicators_complete[df_with_indicators_complete["macd"] > df_with_indicators_complete["macd_signal_diff"]]

In [70]:
test

,complete,currency_pair,time,volume,price_open,price_high,price_low,price_close,macd,macd_signal,macd_signal_diff,sma_200,rsi_14
50,True,EUR_USD,2021-01-03 23:17:00+00:00,30,1.22396,1.22396,1.22365,1.22377,0.000029,0.000003,0.000026,NaN,47.704789
51,True,EUR_USD,2021-01-03 23:18:00+00:00,22,1.22377,1.22390,1.22373,1.22373,0.000016,0.000006,0.000010,NaN,46.465751
52,True,EUR_USD,2021-01-03 23:19:00+00:00,30,1.22375,1.22379,1.22368,1.22375,0.000007,0.000006,0.000001,NaN,47.204127
53,True,EUR_USD,2021-01-03 23:20:00+00:00,37,1.22370,1.22374,1.22348,1.22364,-0.000009,0.000003,-0.000012,NaN,43.639055
62,True,EUR_USD,2021-01-03 23:29:00+00:00,44,1.22404,1.22418,1.22395,1.22410,0.000045,0.000004,0.000041,NaN,60.837381
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9994,True,EUR_USD,2021-01-12 23:36:00+00:00,17,1.22077,1.22080,1.22071,1.22080,0.000024,0.000012,0.000012,1.220598,55.920718
9995,True,EUR_USD,2021-01-12 23:37:00+00:00,10,1.22078,1.22081,1.22078,1.22078,0.000027,0.000015,0.000012,1.220602,54.735902
9996,True,EUR_USD,2021-01-12 23:38:00+00:00,15,1.22080,1.22080,1.22070,1.22074,0.000026,0.000017,0.000009,1.220605,52.347070
9997,True,EUR_USD,2021-01-12 23:39:00+00:00,11,1.22076,1.22085,1.22076,1.22085,0.000034,0.000020,0.000014,1.220610,57.801258
